In [6]:
import pandas as pd
import numpy as np
import sqlite3
con = sqlite3.connect("/Users/skooter/Library/Developer/CoreSimulator/Devices/C71758F6-0B93-44CE-A57B-EDC04C058146/data/Containers/Data/Application/BE701AC6-FC73-4AC0-8573-205C34FE4F0A/Documents/Hewa.db")
df = pd.read_sql_query("SELECT * FROM viewTABLE", con)

In [7]:
print(con)

In [3]:
df

,id,uid,recipeId,isView
0,1,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,1,1
1,2,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,2,1
2,3,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,3,1
3,4,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,4,1
4,5,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,5,1
...,...,...,...,...
245,246,VaipAkcQYfexWPTX16itUC57t1D2,46,0
246,247,VaipAkcQYfexWPTX16itUC57t1D2,47,0
247,248,VaipAkcQYfexWPTX16itUC57t1D2,48,1
248,249,VaipAkcQYfexWPTX16itUC57t1D2,49,0


In [3]:
dataPrep = df[['uid', 'recipeId' , 'isView']]
dataPrep

,uid,recipeId,isView
0,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,1,1
1,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,2,1
2,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,3,1
3,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,4,1
4,x8cKCoJwnqSJZgklwPSBXLtjEgQ2,5,1
...,...,...,...
245,VaipAkcQYfexWPTX16itUC57t1D2,46,0
246,VaipAkcQYfexWPTX16itUC57t1D2,47,0
247,VaipAkcQYfexWPTX16itUC57t1D2,48,1
248,VaipAkcQYfexWPTX16itUC57t1D2,49,0


In [4]:
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

In [5]:
def GetItemItemSim(uid, recipeId):
    recipeCustomerMatrix = csr_matrix(([1]*len(uid), (recipeId, uid)))
    similarity = cosine_similarity(recipeCustomerMatrix)
    return similarity, recipeCustomerMatrix

In [6]:
def get_recommendations_from_similarity(similarity_matrix, SalesItemCustomerMatrix, top_n=10):
    recipeUserItemMatrix = csr_matrix(SalesItemCustomerMatrix.T)
    recipeUserItemScores = recipeUserItemMatrix.dot(similarity_matrix) # sum of similarities to all purchased products
    RecForCust = []
    for uid in range(recipeUserItemScores.shape[0]):
        scores = recipeUserItemScores[uid, :]
        recipeIds = recipeUserItemMatrix.indices[recipeUserItemMatrix.indptr[uid]:
        recipeUserItemMatrix.indptr[uid+1]]
        scores[recipeIds] = -1 # do not recommend already purchased SalesItems
        top_products_ids = np.argsort(scores)[-top_n:][::-1]
        recommendations = pd.DataFrame(
        top_products_ids.reshape(1, -1),
        index=[uid],
        columns=['Top%s' % (i+1) for i in range(top_n)])
        RecForCust.append(recommendations)
    return pd.concat(RecForCust)

In [7]:
def get_recommendations(view_data):
    user_label_encoder = LabelEncoder()
    uids = user_label_encoder.fit_transform(view_data.uid)
    recipe_label_encoder = LabelEncoder()
    recipeIds = recipe_label_encoder.fit_transform(view_data.recipeId)
    # compute recommendations
    similarity_matrix, recipeUserMatrix = GetItemItemSim(uids, recipeIds)
    recommendations = get_recommendations_from_similarity(similarity_matrix, recipeUserMatrix)
    recommendations.index = user_label_encoder.inverse_transform(recommendations.index)
    for i in range(recommendations.shape[1]):
        recommendations.iloc[:, i] = recipe_label_encoder.inverse_transform(recommendations.iloc[:, i])
        return recommendations

In [8]:
recommendations = get_recommendations(dataPrep)
recommendations

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10
F4QoZNVQ5kUgZKfdY7etXSI8AFi2,50,12,22,21,20,19,18,17,16,15
VaipAkcQYfexWPTX16itUC57t1D2,50,12,22,21,20,19,18,17,16,15
bpSTXKotc2Qo87o6JbImdJl4Wk43,50,12,22,21,20,19,18,17,16,15
hb8DLE7if5Se2LNrSLcF2OR1uvy1,50,12,22,21,20,19,18,17,16,15
x8cKCoJwnqSJZgklwPSBXLtjEgQ2,50,12,22,21,20,19,18,17,16,15


In [9]:
def getRecommendations(databaseLocation, input):
    con = sqlite3.connect(databaseLocation)
    df = pd.read_sql_query("SELECT * FROM viewTABLE", con)
    dataPrep = df[['uid', 'recipeId' , 'isView']]
    return get_recommendations(dataPrep).loc[[input]].values.tolist()[0]